In [54]:
import pandas as pd
data = pd.read_json('cladder.json')
data = data.dropna().reset_index(drop=True)

#create column "formal_form" and "graph" that holds the value corresponding to the key "formal_form" at the dict inside column "meta"

data = data.assign(formal_form = lambda x: (x['meta']))
df = data['formal_form'].apply(lambda x: (x.get("formal_form")))
data['formal_form'] = df

data = data.assign(graph = lambda x: (x['reasoning']))
df = data['graph'].apply(lambda x: (x.get("step1")))
data['graph'] = df

data = data.assign(query_type = lambda x: (x['meta']))
df = data['query_type'].apply(lambda x: (x.get("query_type")))
data['query_type'] = df

data = data.assign(info = lambda x: (x['meta']))
df = data['info'].apply(lambda x: (x.get("given_info")))
data['info'] = df
data


,question_id,desc_id,given_info,question,answer,meta,reasoning,formal_form,graph,query_type,info
0,4,alarm-mediation-nde-model0-spec0-q0,"For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.","If we disregard the mediation effect through wife, would husband positively affect alarm clock?",yes,"{'story_id': 'alarm', 'graph_id': 'mediation', 'mediators': ['V2'], 'polarity': True, 'groundtruth': 0.322381664440195, 'query_type': 'nde', 'rung': 3, 'formal_form': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'given_info': {'p(Y | X, V2)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(V2 | X)': [0.741686618881911, 0.23519324071521203]}, 'estimand': '\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'treatment': 'X', 'outcome': 'Y', 'model_id': 0}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'step3': '\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'step4': 'P(Y=1 | X=0, V2=0) = 0.08 P(Y=1 | X=0, V2=1) = 0.54 P(Y=1 | X=1, V2=0) = 0.41 P(Y=1 | X=1, V2=1) = 0.86 P(V2=1 | X=0) = 0.74 P(V2=1 | X=1) = 0.24', 'step5': '0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = 0.32', 'end': '0.32 > 0'}","E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","X->V2,X->Y,V2->Y",nde,"{'p(Y | X, V2)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(V2 | X)': [0.741686618881911, 0.23519324071521203]}"
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.",Will alarm set by husband decrease the chance of ringing alarm?,no,"{'story_id': 'alarm', 'graph_id': 'mediation', 'treated': True, 'result': True, 'polarity': False, 'groundtruth': 0.49506904015332404, 'query_type': 'ate', 'rung': 2, 'formal_form': 'E[Y | do(X = 1)] - E[Y | do(X = 0)]', 'given_info': {'p(Y | X)': [0.26076060984306904, 0.755829649996393]}, 'estimand': 'P(Y=1|X=1) - P(Y=1|X=0)', 'treatment': 'X', 'outcome': 'Y', 'model_id': 1}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y | do(X = 1)] - E[Y | do(X = 0)]', 'step3': 'P(Y=1|X=1) - P(Y=1|X=0)', 'step4': 'P(Y=1 | X=0) = 0.26 P(Y=1 | X=1) = 0.76', 'step5': '0.76 - 0.26 = 0.50', 'end': '0.50 > 0'}",E[Y | do(X = 1)] - E[Y | do(X = 0)],"X->V2,X->Y,V2->Y",ate,"{'p(Y | X)': [0.26076060984306904, 0.755829649996393]}"
2,8,alarm-mediation-marginal-model1-spec1-q0,"The overall probability of alarm set by husband is 77%. For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.",Is ringing alarm more likely than silent alarm overall?,yes,"{'story_id': 'alarm', 'graph_id': 'mediation', 'given_info': {'P(X)': 0.771884206668314, 'P(Y | X)': [0.26076060984306904, 0.755829649996393]}, 'treated': True, 'polarity': True, 'groundtruth': 0.6429672171752431, 'query_type': 'marginal', 'rung': 1, 'formal_form': 'P(Y)', 'treatment': 'X', 'outcome': 'Y', 'model_id': 1}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'P(Y)', 'step3': 'P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0)', 'step4': 'P(X=1) = 0.77 P(Y=1 | X=0) = 0.26 P(Y=1 | X=1) = 0.76', 'step5': '0.77*0.76 - 0.23*0.26 = 0.64', 'end': '0.64 > 0'}",P(Y),"X->V2,X->Y,V2->Y",marginal,"{'P

In [55]:
#erase nonsense sentences
ns = ['nonsense0','nonsense1','nonsense2', 'nonsense3', 'nonsense4', 'nonsense5', 'nonsense6', 'nonsense7', 'nonsense8', 'nonsense9' ]
data = data[data.apply(lambda x: x['meta'].get("story_id") not in ns, axis=1)]

In [56]:
data['query_type'].unique()

array(['nde', 'ate', 'marginal', 'nie', 'ett', 'correlation', 'exp_away',
       'collider_bias', 'det-counterfactual'], dtype=object)

In [57]:
#exchange "formal_form" and "graph" column's variables to their respective "true name" (ex: exchange X to husband, V2 to wife, etc)
def exchange(x, form, aux):
    x = x.get('step0')
    if(aux == 1):
        form = form.get('formal_form')
    elif(aux == 0):
        form = form.get('step1')
    else:
        form = str(form)
    
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    for i in range(len(x)):
        aux = x[i].split(" = ")
        form = form.replace(aux[0],aux[1])
    return form

#create a list of real variable names and their respective representation e.g. [['X','wife'],['Y','husband']]
def list_representation(x):
    x = x.get('step0')
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    aux = []
    for i in range(len(x)):
        aux.append(x[i].split(" = "))
    return aux

#receives a list from list_representation and returns the prompt with it
def prompt_repres(x):
    x = x.get('step0')
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    list_rep = []
    for i in range(len(x)):
        list_rep.append(x[i].split(" = "))
    
    prompt = ""
    for i in range(len(list_rep)):
        prompt += "Use "+list_rep[i][0]+" to represent "+list_rep[i][1]+". "
    return prompt

def create_string(str_list):
    string = ""
    for i in range(str_list):
        string += str_list[i]
    return string

#exchange query_type abbreviation to the whole word
def expand(abb):
    if abb == 'nde':
        return 'natural direct effect'
    elif abb == 'ate':
        return 'average treatment effect'
    elif abb == 'marginal':
        return 'marginal probability'
    elif abb == 'nie':
        return 'natural indirect effect'
    elif abb == 'det-counterfactual':
        return 'normal counterfactual question'
    elif abb == 'ett':
        return 'average treatment effect on treated'
    elif abb == 'correlation':
        return 'conditional probability'
    elif abb == 'exp_away':
        return 'explaining away effect'
    elif abb == 'collider_bias':
        return 'collider bias'
    elif abb == 'backadj':
        return 'backdoor adjustment set'
    return 0
    

#data['formal_form'] = data.apply(lambda x: exchange(x['reasoning'], x['meta'],1), axis=1)
#data['graph'] = data.apply(lambda x: exchange(x['reasoning'], x['reasoning'],0), axis=1)
data['info'] = data.apply(lambda x: exchange(x['reasoning'], x['info'],2), axis=1)
data['query_type'] = data.apply(lambda x: expand(x['query_type']), axis=1)

data

/tmp/ipykernel_201034/442145457.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['info'] = data.apply(lambda x: exchange(x['reasoning'], x['info'],2), axis=1)
/tmp/ipykernel_201034/442145457.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['query_type'] = data.apply(lambda x: expand(x['query_type']), axis=1)


,question_id,desc_id,given_info,question,answer,meta,reasoning,formal_form,graph,query_type,info
0,4,alarm-mediation-nde-model0-spec0-q0,"For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.","If we disregard the mediation effect through wife, would husband positively affect alarm clock?",yes,"{'story_id': 'alarm', 'graph_id': 'mediation', 'mediators': ['V2'], 'polarity': True, 'groundtruth': 0.322381664440195, 'query_type': 'nde', 'rung': 3, 'formal_form': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'given_info': {'p(Y | X, V2)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(V2 | X)': [0.741686618881911, 0.23519324071521203]}, 'estimand': '\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'treatment': 'X', 'outcome': 'Y', 'model_id': 0}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'step3': '\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'step4': 'P(Y=1 | X=0, V2=0) = 0.08 P(Y=1 | X=0, V2=1) = 0.54 P(Y=1 | X=1, V2=0) = 0.41 P(Y=1 | X=1, V2=1) = 0.86 P(V2=1 | X=0) = 0.74 P(V2=1 | X=1) = 0.24', 'step5': '0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = 0.32', 'end': '0.32 > 0'}","E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","X->V2,X->Y,V2->Y",natural direct effect,"{'p(alarm clock | husband, wife)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(wife | husband)': [0.741686618881911, 0.23519324071521203]}"
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.",Will alarm set by husband decrease the chance of ringing alarm?,no,"{'story_id': 'alarm', 'graph_id': 'mediation', 'treated': True, 'result': True, 'polarity': False, 'groundtruth': 0.49506904015332404, 'query_type': 'ate', 'rung': 2, 'formal_form': 'E[Y | do(X = 1)] - E[Y | do(X = 0)]', 'given_info': {'p(Y | X)': [0.26076060984306904, 0.755829649996393]}, 'estimand': 'P(Y=1|X=1) - P(Y=1|X=0)', 'treatment': 'X', 'outcome': 'Y', 'model_id': 1}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y | do(X = 1)] - E[Y | do(X = 0)]', 'step3': 'P(Y=1|X=1) - P(Y=1|X=0)', 'step4': 'P(Y=1 | X=0) = 0.26 P(Y=1 | X=1) = 0.76', 'step5': '0.76 - 0.26 = 0.50', 'end': '0.50 > 0'}",E[Y | do(X = 1)] - E[Y | do(X = 0)],"X->V2,X->Y,V2->Y",average treatment effect,"{'p(alarm clock | husband)': [0.26076060984306904, 0.755829649996393]}"
2,8,alarm-mediation-marginal-model1-spec1-q0,"The overall probability of alarm set by husband is 77%. For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.",Is ringing alarm more likely than silent alarm overall?,yes,"{'story_id': 'alarm', 'graph_id': 'mediation', 'given_info': {'P(X)': 0.771884206668314, 'P(Y | X)': [0.26076060984306904, 0.755829649996393]}, 'treated': True, 'polarity': True, 'groundtruth': 0.6429672171752431, 'query_type': 'marginal', 'rung': 1, 'formal_form': 'P(Y)', 'treatment': 'X', 'outcome': 'Y', 'model_id': 1}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'P(Y)', 'step3': 'P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0)', 'step4': 'P(X=1) = 0.77 P(Y=1 | X=0) = 0.26 P(Y=1 | X=1) = 0.76', 'step5': '0.77*0.

In [58]:
pd.set_option('display.max_colwidth', None)

In [59]:
prompt_repres(list_representation(data.iloc[0]['reasoning']))


AttributeError: 'list' object has no attribute 'get'

In [61]:
#create columnn with complete prompt to be given to the LLM
paraphrases = [
            "Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships",
            "Think of a self-contained, hypothetical setting with just the specified conditions, and devoid of any unknown factors or causal connections",
            "Consider a self-contained, hypothetical world with solely the mentioned conditions, and is free from any hidden factors or cause-and-effect relationships",
            "Imagine a self-contained, hypothetical setting with merely the stated conditions, and absent any unmentioned factors or causative links",
            "Think of a self-contained, hypothetical world with only the given conditions, and is void of any unknown factors or causative relationships",
        ]

prompt_end0 = "Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. "
prompt_end1 = "The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed. Step 4) Extract all the available data. Your answer should contain nothing but marginal probabilities and conditional probabilities in the form 'P(...)=...' or 'P(...|...)=...', each probability being separated by a semicolon. Stick to the previously mentioned denotations for the variables. Step 5) Given all the information above, deduce the estimand using skills such as do-calculus, counterfactual prediction, and the basics of probabilities. Answer step by step. Step 6) Insert the relevant data in Step 4 into the estimand, perform basic arithmetic calculations, and derive the final answer. There is an identifiable answer. Answer step by step. \nBased on all the reasoning above, output one word to answer the initial question."

data['prompt'] = data.apply(lambda x: (paraphrases[0]+x['given_info']+x['question']+prompt_end0+prompt_repres(x['reasoning'])+prompt_end1),axis=1)
df = data[['prompt', 'formal_form', 'graph', 'query_type']]
df

/tmp/ipykernel_201034/948927871.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['prompt'] = data.apply(lambda x: (paraphrases[0]+x['given_info']+x['question']+prompt_end0+prompt_repres(x['reasoning'])+prompt_end1),axis=1)


,prompt,formal_form,graph,query_type
0,"Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationshipsFor husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.If we disregard the mediation effect through wife, would husband positively affect alarm clock?Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed. Step 4) Extract all the available data. Your answer should contain nothing but marginal probabilities and conditional probabilities in the form 'P(...)=...' or 'P(...|...)=...', each probability being separated by a semicolon. Stick to the previously mentioned denotations for the variables. Step 5) Given all the information above, deduce the estimand using skills such as do-calculus, counterfactual prediction, and the basics of probabilities. Answer step by step. Step 6) Insert the relevant data in Step 4 into the estimand, perform basic arithmetic calculations, and derive the final answer. There is an identifiable answer. Answer step by step. \nBased on all the reasoning above, output one word to answer the initial question.","E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","X->V2,X->Y,V2->Y",natural direct effect
1,"Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationshipsFor husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.Will alarm set by husband decrease the chance of ringing alarm?Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notatio

In [35]:
list_representation(data.iloc[0]['reasoning'])


[['X', 'husband'], ['V2', 'wife'], ['Y', 'alarm clock']]

In [65]:
#transform the dataset into an "assistant type"

df = df.assign(text = lambda x: ('### Human: ' + x['prompt'] + '### Assistant: Step 1) ' + x['graph'] + '. Step 2) ' + 
                                            x['query_type'] + '. Step 3) ' + x['formal_form'] + '. Step 4) ' +
                                            ))

final_dataset = df[['text']]
final_dataset

,text
0,"### Human: Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationshipsFor husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.If we disregard the mediation effect through wife, would husband positively affect alarm clock?Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed. Step 4) Extract all the available data. Your answer should contain nothing but marginal probabilities and conditional probabilities in the form 'P(...)=...' or 'P(...|...)=...', each probability being separated by a semicolon. Stick to the previously mentioned denotations for the variables. Step 5) Given all the information above, deduce the estimand using skills such as do-calculus, counterfactual prediction, and the basics of probabilities. Answer step by step. Step 6) Insert the relevant data in Step 4 into the estimand, perform basic arithmetic calculations, and derive the final answer. There is an identifiable answer. Answer step by step. \nBased on all the reasoning above, output one word to answer the initial question.### Assistant: Step 1) X->V2,X->Y,V2->Y. Step 2) natural direct effect. Step 3) E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]."
1,"### Human: Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationshipsFor husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.Will alarm set by husband decrease the chance of ringing alarm?Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, u

In [62]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
#import transformers
#import torch

#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

#pipeline = transformers.pipeline(
#    "text-generation",
#    model=model_id,
#    model_kwargs={"torch_dtype": torch.bfloat16},
#    device_map="auto",
#)

#messages = [
#    {"role": "system", "content": "You are a chatbot"},
#    {"role": "user", "content": "Who are you?"},
#]

#prompt = pipeline.tokenizer.apply_chat_template(
#        messages, 
#        tokenize=False, 
#        add_generation_prompt=True
#)

#terminators = [
#    pipeline.tokenizer.eos_token_id,
#    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
#]

#outputs = pipeline(
#    prompt,
#    max_new_tokens=256,
#    eos_token_id=terminators,
#    do_sample=True,
#    temperature=0.6,
#    top_p=0.9,
#)
#print(outputs[0]["generated_text"][len(prompt):])


In [9]:
#pipeline("Hey how are you doing today?")

In [10]:
#pipeline.device

In [11]:
#round(0.23519324071521203,2)

In [12]:
#the following code is from this website:
#https://deci.ai/blog/fine-tune-llama-2-with-lora-for-question-answering/

In [63]:
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [66]:
# Dataset
#Transform dataset from DataFrame to Dataset
import datasets
from datasets import Dataset, DatasetDict

final_dataset = final_dataset.sample(frac=1).reset_index(drop=True)
final_dataset = Dataset.from_pandas(final_dataset)
split_form = final_dataset.train_test_split(train_size=0.9, seed=20)
training_data = split_form['train']

# Model and tokenizer names
#base_model_name = "openai-community/gpt2-large"
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
refined_model = "/media/data/bazaluk/cladder_llama3_model"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [67]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="/media/data/bazaluk/results_llama3",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(base_model)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 

In [70]:
df.iloc[0]

prompt                                                                                                                                     Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationshipsFor husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.If we disregard the mediation effect through wife, would husband positively affect alarm clock?Guidance: Address the question

In [75]:
query = "Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationshipsFor husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.If we disregard the mediation effect through wife, would husband positively affect alarm clock?Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed. Step 4) Extract all the available data. Your answer should contain nothing but marginal probabilities and conditional probabilities in the form 'P(...)=...' or 'P(...|...)=...', each probability being separated by a semicolon. Stick to the previously mentioned denotations for the variables. Step 5) Given all the information above, deduce the estimand using skills such as do-calculus, counterfactual prediction, and the basics of probabilities. Answer step by step. Step 6) Insert the relevant data in Step 4 into the estimand, perform basic arithmetic calculations, and derive the final answer. There is an identifiable answer. Answer step by step. \nBased on all the reasoning above, output one word to answer the initial question:"
text_gen = pipeline(task="text-generation", model=base_model_name, tokenizer=llama_tokenizer, max_length=700)
#output = text_gen(f"<s>[INST] {query} [/INST]")
#print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [76]:
text_gen(query)

[{'generated_text': 'Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationshipsFor husbands that don\'t set the alarm and wives that don\'t set the alarm, the probability of ringing alarm is 8%. For husbands that don\'t set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don\'t set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don\'t set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.If we disregard the mediation effect through wife, would husband positively affect alarm clock?Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the rela